In [ ]:
#Libraries 
import numpy as np 
import pandas as pd 
import os,unicodedata, re, io, time, gc, warnings
#warnings.filterwarnings("ignore")
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
%matplotlib inline
import math
import random
import string
import re
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import TextVectorization

In [ ]:
df_1 = pd.read_excel('/content/final_new_dataset_1034K.xlsx' ,sheet_name = 0 , names=["har","eng"]) #, index_col = 0
#df_2 = pd.read_excel('/content/final_new_dataset_1034K.xlsx' ,sheet_name = 1 , names=["har","eng"]) #, index_col = 0


In [ ]:
df=df_1

In [ ]:
#df = pd.concat([df_1,df_2], ignore_index=True)

In [ ]:
df

,har,eng
0,अँधोड़,Strong wind
1,अंगार,coal
2,अंघाई,Mischief
3,अंडास,Obstacle
4,अंतिम,the last
...,...,...
1034171,छोटा सा छोरा,little boy
1034172,कोए मैंने ना रोक सकै,no one can stop me
1034173,ना ए मैं शरमाऊं,no i'm ashamed
1034174,एक नारी इस्सी देखि,saw a woman like this


In [ ]:
#df = pd.read_excel('/content/final_new_dataset_604K.xlsx', names=["har","eng"])

In [ ]:
# df = df.astype(str)
#df

In [ ]:
text_pairs=[]

for i,j in zip(df["har"].tolist(), df["eng"].tolist()):
    j = "[start] " + j + " [end]"
    text_pairs.append((i,j))

In [ ]:
for i in range(5):
    print(random.choice(text_pairs))

('माळ', '[start] Beehive [end]')
('यो मेरा बापु हाई', '[start] He is my father. [end]')
('शिवलाल ', '[start] shivlal [end]')
('बीजना', '[start] Hand fan [end]')
('म्हारा बाप साथ साल का से', '[start] My father is 60 years old. [end]')


In [ ]:
random.shuffle(text_pairs)             #randomly shuffl kiya jitn bhi pairs diye
num_val_samples = int(0.15 * len(text_pairs))     #validation set mei daln ke liy textpair*0.15 num kiye
num_train_samples = len(text_pairs) - 2 * num_val_samples        #how many in train sample
train_pairs = text_pairs[:num_train_samples]                 #defined train_pairs
val_pairs = text_pairs[num_train_samples : num_train_samples + num_val_samples]            #val pair aftr train vala datast
test_pairs = text_pairs[num_train_samples + num_val_samples :]              # uske baad ka bachaa hua to test pair
print(train_pairs)
print(f"{len(text_pairs)} total pairs")
print(f"{len(train_pairs)} training pairs")
print(f"{len(val_pairs)} validation pairs")
print(f"{len(test_pairs)} test pairs")

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



## Vectorizing the text data

We'll use two instances of the `TextVectorization` layer to vectorize the text
data (one for English and one for Spanish),
that is to say, to turn the original strings into integer sequences
where each integer represents the index of a word in a vocabulary.

The English layer will use the default string standardization (strip punctuation characters)
and splitting scheme (split on whitespace), while
the Spanish layer will use a custom standardization, where we add the character
`"¿"` to the set of punctuation characters to be stripped.

Note: in a production-grade machine translation model, I would not recommend
stripping the punctuation characters in either language. Instead, I would recommend turning
each punctuation character into its own token,
which you could achieve by providing a custom `split` function to the `TextVectorization` layer.

In [ ]:
strip_chars = string.punctuation + "¿"
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")

vocab_size = 3200
sequence_length = 320
batch_size = 128


def custom_standardization(input_string):
    lowercase = tf.strings.lower(input_string)
    return tf.strings.regex_replace(lowercase, "[%s]" % re.escape(strip_chars), "")


eng_vectorization = TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length,
)
har_vectorization = TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length + 1,
    standardize=custom_standardization,
)
train_eng_texts = [pair[0] for pair in train_pairs]
train_har_texts = [pair[1] for pair in train_pairs]
print(train_eng_texts)
eng_vectorization.adapt(train_eng_texts)
har_vectorization.adapt(train_har_texts)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
train_har_texts, train_eng_texts 

(['[start] Headache [end]',
  '[start] day [end]',
  '[start] The iron-spoon [end]',
  '[start] My grandfather still sleeps there at night. [end]',
  '[start] invitation [end]',
  '[start] rabbit [end]',
  '[start] deedaaree [end]',
  '[start] Deep [end]',
  '[start] Kushmalta [end]',
  '[start] Sujansingh [end]',
  '[start] What can I do? [end]',
  '[start] mild [end]',
  '[start] Lalita [end]',
  '[start] haradeee [end]',
  '[start] Both [end]',
  '[start] good [end]',
  '[start] Apply [end]',
  '[start] erect [end]',
  '[start] tirakha [end]',
  '[start] trust [end]',
  '[start] Beehive [end]',
  '[start] Why do you drive the car? [end]',
  '[start] The kebabs will be eaten. [end]',
  '[start] nightmare [end]',
  '[start] Hemraj  [end]',
  '[start] Small Children [end]',
  '[start] ransingh [end]',
  '[start] What are your plans for Holi?  [end]',
  '[start] I am a vegetarian [end]',
  '[start] Thakaria [end]',
  '[start] Brain [end]',
  '[start] dharamveer [end]',
  '[start] to mee

In [ ]:
def format_dataset(eng, har):
    eng = eng_vectorization(eng)
    har = har_vectorization(har)
    return (
        {
            "encoder_inputs": eng,
            "decoder_inputs": har[:, :-1],
        },
        har[:, 1:],
    )


def make_dataset(pairs):
    eng_texts, har_texts = zip(*pairs)
    eng_texts = list(eng_texts)
    har_texts = list(har_texts)
    dataset = tf.data.Dataset.from_tensor_slices((eng_texts, har_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset)
    return dataset.shuffle(2048).prefetch(16).cache()


train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

Let's take a quick look at the sequence shapes
(we have batches of 64 pairs, and all sequences are 20 steps long):

In [ ]:
train_ds

<CacheDataset element_spec=({'encoder_inputs': TensorSpec(shape=(None, 320), dtype=tf.int64, name=None), 'decoder_inputs': TensorSpec(shape=(None, 320), dtype=tf.int64, name=None)}, TensorSpec(shape=(None, 320), dtype=tf.int64, name=None))>

In [ ]:
for inputs, targets in train_ds.take(1):
    print(f'inputs["encoder_inputs"].shape: {inputs["encoder_inputs"].shape}')
    print(f'inputs["decoder_inputs"].shape: {inputs["decoder_inputs"].shape}')
    print(f"targets.shape: {targets.shape}")

inputs["encoder_inputs"].shape: (128, 320)
inputs["decoder_inputs"].shape: (128, 320)
targets.shape: (128, 320)


In [ ]:
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super(TransformerEncoder, self).__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.dense_proj = keras.Sequential(
            [
                layers.Dense(dense_dim, activation="relu"),
                layers.Dense(embed_dim),
            ]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.supports_masking = True

    def call(self, inputs, mask=None):
        if mask is not None:
            padding_mask = tf.cast(mask[:, tf.newaxis, tf.newaxis, :], dtype="int32")
        attention_output = self.attention(
            query=inputs, value=inputs, key=inputs, attention_mask=padding_mask
        )
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)
    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "dense_dim": self.dense_dim,
            "num_heads": self.num_heads,
        })
        return config


class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, vocab_size, embed_dim, **kwargs):
        super(PositionalEmbedding, self).__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=vocab_size, output_dim=embed_dim
        )
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=embed_dim
        )
        self.sequence_length = sequence_length
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)
    def get_config(self):
        config = super().get_config()
        config.update({
            "sequence_length": self.sequence_length,
            "vocab_size": self.vocab_size,
            "embed_dim": self.embed_dim,
        })
        return config


class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, latent_dim, num_heads, **kwargs):
        super(TransformerDecoder, self).__init__(**kwargs)
        self.embed_dim = embed_dim
        self.latent_dim = latent_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.attention_2 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.dense_proj = keras.Sequential(
            [
                layers.Dense(latent_dim, activation="relu"),
                layers.Dense(embed_dim),
            ]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True

    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(mask[:, tf.newaxis, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)

        attention_output_1 = self.attention_1(
            query=inputs, value=inputs, key=inputs, attention_mask=causal_mask
        )
        out_1 = self.layernorm_1(inputs + attention_output_1)

        attention_output_2 = self.attention_2(
            query=out_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
        )
        out_2 = self.layernorm_2(out_1 + attention_output_2)

        proj_output = self.dense_proj(out_2)
        return self.layernorm_3(out_2 + proj_output)

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1), tf.constant([1, 1], dtype=tf.int32)],
            axis=0,
        )
        return tf.tile(mask, mult)
    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "latent_dim": self.latent_dim,
            "num_heads": self.num_heads,
        })
        return config


In [ ]:
config

Available objects for config:
     AliasManager
     ColabHistoryManager
     DisplayFormatter
     IPCompleter
     IPKernelApp
     InlineBackend
     LoggingMagics
     MagicsManager
     OSMagics
     PrefilterManager
     ScriptMagics
     Shell
     StoreMagics


In [ ]:
embed_dim = 128
latent_dim = 1024
num_heads = 8

encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="encoder_inputs")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)
encoder_outputs = TransformerEncoder(embed_dim, latent_dim, num_heads)(x)
encoder = keras.Model(encoder_inputs, encoder_outputs)

decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="decoder_inputs")
encoded_seq_inputs = keras.Input(shape=(None, embed_dim), name="decoder_state_inputs")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)
x = TransformerDecoder(embed_dim, latent_dim, num_heads)(x, encoded_seq_inputs)
x = layers.Dropout(0.5)(x)
decoder_outputs = layers.Dense(vocab_size, activation="softmax")(x)
decoder = keras.Model([decoder_inputs, encoded_seq_inputs], decoder_outputs)

decoder_outputs = decoder([decoder_inputs, encoder_outputs])
transformer = keras.Model(
    [encoder_inputs, decoder_inputs], decoder_outputs, name="transformer"
)

In [ ]:
epochs = 2

transformer.summary()
transformer.compile(
    "rmsprop", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)
transformer.fit(train_ds, epochs=epochs, validation_data=val_ds)

Model: "transformer"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 encoder_inputs (InputLayer)    [(None, None)]       0           []                               
                                                                                                  
 positional_embedding (Position  (None, None, 128)   450560      ['encoder_inputs[0][0]']         
 alEmbedding)                                                                                     
                                                                                                  
 decoder_inputs (InputLayer)    [(None, None)]       0           []                               
                                                                                                  
 transformer_encoder (Transform  (None, None, 128)   791296      ['positional_embedding[

## Decoding test sentences

Finally, let's demonstrate how to translate brand new English sentences.
We simply feed into the model the vectorized English sentence
as well as the target token `"[start]"`, then we repeatedly generated the next token, until
we hit the token `"[end]"`.

In [ ]:
har_vocab = har_vectorization.get_vocabulary()
har_index_lookup = dict(zip(range(len(har_vocab)), har_vocab))
max_decoded_sentence_length = 20


def decode_sequence(input_sentence):
    tokenized_input_sentence = eng_vectorization([input_sentence])
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = har_vectorization([decoded_sentence])[:, :-1]
        predictions = transformer([tokenized_input_sentence, tokenized_target_sentence])

        sampled_token_index = np.argmax(predictions[0, i, :])
        sampled_token = har_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token

        if sampled_token == "[end]":
            break
    return decoded_sentence


test_eng_texts = [pair[0] for pair in test_pairs]
print(test_eng_texts)
for _ in range(100):
    input_sentence = random.choice(test_eng_texts)
    translated = decode_sequence(input_sentence)
    print("Har :-" + input_sentence)
    print("Eng :-" + translated)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Har :-अंजुबाला 
Eng :-[start] anjubala [end]
Har :-मन्ने बेरा कद दो गे?
Eng :-[start] when will you get back to me [end]
Har :-अळगोजा
Eng :-[start] flute [end]
Har :-माने देदे या बोतल
Eng :-[start] give me this bottle [end]
Har :-सतीश 
Eng :-[start] satish [end]
Har :-गोधूरामम गुट्टी
Eng :-[start] godhuramam gutti [end]
Har :-अकेला 
Eng :-[start] alone [end]
Har :-शिकायत
Eng :-[start] complaint [end]
Har :-मोकळा
Eng :-[start] enough [end]
Har :-चांदबीर 
Eng :-[start] chandbir [end]
Har :-माड़ा-मोटा
Eng :-[start] a little bit [end]
Har :-पचास
Eng :-[start] fifty [end]
Har :-बछया
Eng :-[start] cows female offspring [end]
Har :-महोदय
Eng :-[start] head [end]
Har :-चमासा
Eng :-[start] brotherinlaw [end]
Har :-जल्दी
Eng :-[start] early [end]
Har :-धोरे सिक खरा किथ से ?
Eng :-[start] where is the nearest akhaara [end]
Har :-इमरजेंसी होरी हा
Eng :-[start] it is an emergency [end]
Har :-के मैं डाकटर ने दिखन तय जाऊ?
Eng :-[start] should i go to see a doctor [end]
Har :-कासण
Eng :-[start] utensil

In [ ]:
decode_sequence('राम ने रोटी खाई से।') #right any sentence in english राम ने रोटी खाई से।   अरे, यो फिलम कद से खतम होगी?

'[start] ram ate the bread [end]'

In [ ]:
from joblib import dump, load
# from pickle import dump, load

In [ ]:
eng_vectorization

In [ ]:
dump({'config': eng_vectorization.get_config(),
             'weights': eng_vectorization.get_weights()}
            , open("eng_tokens.pkl2", "wb"))
dump({'config': har_vectorization.get_config(),
             'weights': har_vectorization.get_weights()}
            , open("har_tokens.pkl2", "wb"))

In [ ]:
from_disk = load(open("eng_tokens.pkl2", "rb"))

In [ ]:
from_disk = load(open("eng_tokens.pkl2", "rb"))
from_disk['config']

{'name': 'text_vectorization',
 'trainable': True,
 'batch_input_shape': (None,),
 'dtype': 'string',
 'max_tokens': 3200,
 'standardize': 'lower_and_strip_punctuation',
 'split': 'whitespace',
 'ngrams': None,
 'output_mode': 'int',
 'output_sequence_length': 320,
 'pad_to_max_tokens': False,
 'sparse': False,
 'ragged': False,
 'vocabulary': None,
 'idf_weights': None}

In [ ]:
new_vectorizer = TextVectorization(max_tokens=from_disk['config']['max_tokens'],
                                          output_mode='int',
                                          output_sequence_length=from_disk['config']['output_sequence_length'])

In [ ]:
new_vectorizer.adapt(tf.data.Dataset.from_tensor_slices(["xyz"]))

In [ ]:
new_vectorizer.set_weights(from_disk['weights'])
print(new_vectorizer("this"))

tf.Tensor(
[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0], shape=(320,), dtype=int64)


In [ ]:
transformer.save('model14_9_w.h5')

In [ ]:
transformer = keras.models.load_model('/content/model14_9_w.h5' , custom_objects={'PositionalEmbedding': PositionalEmbedding ,
                                                                      'TransformerEncoder': TransformerEncoder,
                                                                      'TransformerDecoder': TransformerDecoder})

In [ ]:
#from keras.models import Sequential, load_model

In [ ]:
#from tensorflow import keras